In [1]:
from transformers import YolosImageProcessor, YolosForObjectDetection
from PIL import Image
import torch
import requests

/home/sgarg26/Documents/p5/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
image = Image.open('data/blend.png')

In [15]:
model = YolosForObjectDetection.from_pretrained('hustvl/yolos-tiny')
image_processor = YolosImageProcessor.from_pretrained("hustvl/yolos-tiny")

In [ ]:
inputs = image_processor(images=image, return_tensors="pt")
outputs = model(**inputs)

In [6]:
logits = outputs.logits
bboxes = outputs.pred_boxes

In [7]:
target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.1, target_sizes=target_sizes)[0]
for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(
        f"Detected {model.config.id2label[label.item()]} with confidence "
        f"{round(score.item(), 3)} at location {box}"
    )

Detected frisbee with confidence 0.15 at location [486.1, 401.41, 712.35, 588.89]
Detected skis with confidence 0.134 at location [371.52, 44.93, 461.61, 371.32]
Detected tv with confidence 0.175 at location [230.56, 0.37, 445.96, 72.41]
Detected frisbee with confidence 0.351 at location [513.18, 185.19, 709.96, 311.59]
Detected scissors with confidence 0.109 at location [838.38, 102.87, 985.34, 213.84]


In [2]:
from dotenv import load_dotenv
from roboflow import Roboflow
import os
_ = load_dotenv()

In [3]:

from roboflow import Roboflow
rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
project = rf.workspace("vex2").project("detect_rings")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.58, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to detect_rings-3 in yolov8:: 100%|██████████| 170/170 [00:00<00:00, 10151.94it/s]


In [4]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")

In [5]:
results = model.train(data="/home/sgarg26/Documents/p5/yolo/detect_rings-2/data.yaml", epochs=100)

New https://pypi.org/project/ultralytics/8.2.60 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.58 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/sgarg26/Documents/p5/yolo/detect_rings-2/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_

train: Scanning /home/sgarg26/Documents/p5/yolo/detect_rings-2/train/labels.cache... 36 images, 0 backgrounds, 0 corrupt: 100%|██████████| 36/36 [00:00<?, ?it/s]
val: Scanning /home/sgarg26/Documents/p5/yolo/detect_rings-2/valid/labels.cache... 2 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2/2 [00:00<?, ?it/s]


Plotting labels to /home/sgarg26/Documents/p5/runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to /home/sgarg26/Documents/p5/runs/detect/train
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/100      2.24G      1.349      2.973      1.493          8        640: 100%|██████████| 3/3 [00:01<00:00,  2.29it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  6.39it/s]

                   all          2          2    0.00333          1      0.995      0.577


...

    100/100      2.14G     0.6762     0.6168     0.9417         16        640: 100%|██████████| 3/3 [00:00<00:00, 10.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 30.06it/s]

                   all          2          2      0.983          1      0.995      0.995



100 epochs completed in 0.032 hours.
Optimizer stripped from /home/sgarg26/Documents/p5/runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from /home/sgarg26/Documents/p5/runs/detect/train/weights/best.pt, 6.3MB

Validating /home/sgarg26/Documents/p5/runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.58 🚀 Python-3.11.9 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3070, 8191MiB)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 26.80it/s]


                   all          2          2      0.983          1      0.995      0.995
Speed: 1.0ms preprocess, 11.0ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /home/sgarg26/Documents/p5/runs/detect/train


In [2]:
from ultralytics import YOLO

model = YOLO("best.pt") # YOLO will create a file with "best" and "last weights"

In [6]:
model.export(format="onnx", optimize=True) # Can be exported to use elsewhere. Don't need to worry about platform specifically

Ultralytics YOLOv8.2.58 🚀 Python-3.11.9 torch-2.3.1+cu121 CPU (AMD Ryzen 5 5600 6-Core Processor)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/home/sgarg26/Documents/p5/runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)

ONNX: starting export with onnx 1.16.1 opset 17...
ONNX: export success ✅ 0.4s, saved as '/home/sgarg26/Documents/p5/runs/detect/train/weights/best.onnx' (11.7 MB)

Export complete (1.8s)
Results saved to /home/sgarg26/Documents/p5/runs/detect/train/weights
Predict:         yolo predict task=detect model=/home/sgarg26/Documents/p5/runs/detect/train/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=/home/sgarg26/Documents/p5/runs/detect/train/weights/best.onnx imgsz=640 data=/home/sgarg26/Documents/p5/yolo/detect_rings-2/data.yaml  
Visualize:       https://netron.app


'/home/sgarg26/Documents/p5/runs/detect/train/weights/best.onnx'